# Working out database loading

A scratchpad for updating the database, which might eventually become a little more automated.

In [1]:
from spotclient import Client
import models

db = models.Database()
session = models.get_session(create_all=True)

c = Client()
ab = c.allbirds()
latest = ab[0]
print(latest['name'])

JQBX :: Conference of the Birds 2021-04-22


In [ ]:
# update the database with the latest material
playlist = db.insert_playlist_from_json(session, latest)
db.rebuild_fts(session)
session.commit()


In [2]:
db.insert_playlist_from_json(session, ab[1])
db.rebuild_fts(session)
session.commit()


In [3]:
p = session.query(models.Playlist).filter(models.Playlist.name == 'JQBX :: Conference of the Birds 2021-04-15').scalar()
p

Playlist(3YUtvIkBhG81y4hNPXOl2M)

In [ ]:
session.delete(p)
session.commit()

In [4]:
for i,t in enumerate(p.tracks):
    print(i,t)

0 Fyah by Soul Rebel Sound, Pressure Busspipe
1 Longe tão perto by Os Vips
2 Greenpath by Christopher Larkin
3 Miles' Theme by Shirley Scott
4 Maghostut Two by Ernest Dawkins, New Horizons Ensemble
5 Treat 'Em Right by Chubb Rock
6 Mystic Bounce by Madlib
7 The Colors That You Bring by Damon Locks, Black Monument Ensemble
8 Death Sting Me Blues by Sara Martin
9 I Need Never Get Old - Live by Nathaniel Rateliff & The Night Sweats
10 Funky World Part 1 by Silky Vincent
11 Loving You Is on My Mind by The Meters
12 Matter of Time by Sharon Jones & The Dap-Kings
13 Yuh Body - Remix by Busy Signal, ZJ Sparks, Loud City
14 Original Nuttah 25 - Benny L Remix by Uk Apache, SHY FX, Benny L
15 Spruce Tops by Bibio
16 Easy Going by Señora
17 Love the One You're With by The Isley Brothers
18 Pass The Dutchie by Musical Youth
19 Gimme Some More by Labi Siffre
20 Let Me Try by MC5
21 (I'm Afraid) The Masquerade Is Over by Etta James
22 They Say It's Wonderful by John Coltrane, Johnny Hartman
23 It's 

In [ ]:
nt = p.tracks[:35] + p.tracks[36:]

In [ ]:
p.tracks = nt

In [ ]:
session.rollback()

In [ ]:
list(p)

In [ ]:
#!cat create_and_populate_track_search.sql | sqlite3 birdnest.db
FTS_INSERT_SQL = '''
insert into track_search (track_id, track, artist, album) 
    select t.track_id,
           t.name track,
           GROUP_CONCAT(a.name,';') artist,
           album.name album
    from 
        playlist p,
        playlist_track pt,
        track t,
        track_artist ta,
        artist a,
        album 
    where t.track_id = ta.track_id 
         and ta.artist_id = a.artist_id
         and p.playlist_id = pt.playlist_id
         and pt.track_id = t.track_id
         and t.album_id = album.album_id
         and p.playlist_id = :playlist_id
    group by t.track_id, t.name'''


In [ ]:
r = session.execute(FTS_INSERT_SQL, params={'playlist_id': 50})

In [ ]:
session.commit()

In [ ]:

def load_all_playlists(session):
    c = Client()
    ab = c.allbirds()
    for p in reversed(ab):
        db.insert_playlist_from_json(session,p)
        print(p['name'])
    fill_in_albums(session) 
        

In [ ]:
track.album.name